In [1]:
from __future__ import print_function
import mxnet as mx
import numpy as np
from mxnet import nd, autograd, gluon
import csv
import pandas as pd

ctx = mx.cpu()

# Define some variables

# How many samples to process at once when training
batch_size = 64

# How many classes of gestures you want to have (has to correlate with your data, including "no-gesture" class)
num_outputs = 7

# How many features in each data sample (3 for acceleration, 3 for gravity for a total of 6)
num_features_in_sample = 6

# How many readings qualify as one complete gesture
num_datapoints_in_gesture = 40

# How many total readings are in a single gesture
num_readings_in_gesture = num_features_in_sample * num_datapoints_in_gesture

In [6]:
# Load the training data CSV file
training_data_reader = pd.read_csv('training-data-complete.csv',parse_dates=['time'])
training_data_reader.set_index('time')

# Load the testing data CSV file
testing_data_reader = pd.read_csv('testing-data.csv',parse_dates=['time'])
testing_data_reader.set_index('time')

# Do some data massaging to get it to format that the network needs
training_data_matrix = training_data_reader.as_matrix()
testing_data_matrix = testing_data_reader.as_matrix()

training_data = np.asarray(training_data_matrix[:,0:num_features_in_sample], dtype=np.float64)
testing_data = np.asarray(testing_data_matrix[:,0:num_features_in_sample], dtype=np.float64)

training_data = training_data.reshape(-1, num_datapoints_in_gesture, num_features_in_sample)
testing_data = testing_data.reshape(-1, num_datapoints_in_gesture, num_features_in_sample)

training_labels = training_data_matrix[:,num_features_in_sample + 1]
testing_labels = testing_data_matrix[:,num_features_in_sample + 1]

# To prepare training and testing labels, we cut off the dash and the digits at the end of the label name, so "fan-01" becomes "fan"
training_labels = [x[0:-3] for x in training_labels]
testing_labels = [x[0:-3] for x in testing_labels]

class_dict = {
    'tv-poke': 1, 
    'fan-one-circle': 5, 
    'fan-two-circles':6, 
    'shutter-right-left':2, 
    'no-gesture':0, 
    'letter-m':3, 
    'letter-y': 4
}

# we then turn each label into a corresponding number
training_numerical_labels = np.asarray([class_dict[x] for x in training_labels])
testing_numerical_labels = np.asarray([class_dict[x] for x in testing_labels])

# now, get a list of all labels
training_labels = training_numerical_labels[0:-1:num_datapoints_in_gesture]
testing_labels = testing_numerical_labels[0:-1:num_datapoints_in_gesture]

In [7]:
# Prepare the datasets from the training data and labels, do the same for testing data and labels
training_dataset_array = mx.gluon.data.ArrayDataset(mx.nd.array(training_data), mx.nd.array(training_labels))
testing_dataset_array = mx.gluon.data.ArrayDataset(mx.nd.array(testing_data), mx.nd.array(testing_labels))

# Load the data, shuffling the training data and using the batch size mentioned earlier
training_data = mx.gluon.data.DataLoader(training_dataset_array, batch_size=batch_size, shuffle=True)
testing_data = mx.gluon.data.DataLoader(testing_dataset_array, batch_size=batch_size, shuffle=False)

In [8]:
# Construct a multi layer perceptron, with some hidden layers
# define number of neurons in each hidden layer
num_hidden = 128

net = gluon.nn.Sequential()
with net.name_scope():
    ###########################
    # Adding first hidden layer
    ###########################
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    ###########################
    # Adding dropout with rate .5 to the first hidden layer
    ###########################
    net.add(gluon.nn.Dropout(.5))

    ###########################
    # Adding first hidden layer
    ###########################
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    ###########################
    # Adding dropout with rate .5 to the second hidden layer
    ###########################
    net.add(gluon.nn.Dropout(.5))

    ###########################
    # Adding the output layer
    ###########################
    net.add(gluon.nn.Dense(num_outputs))

In [9]:
# Initialize the neural network
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx, force_reinit=True)
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
# Define the trainer
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .05})

# Create a function for accuracy evaluation
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx).reshape((-1, num_readings_in_gesture))
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [10]:
# Define some training parameters

# How many epochs to run the training (try various numbers)
epochs = 120

smoothing_constant = .01

for e in range(epochs):
    for i, (data, label) in enumerate(training_data):
        data = data.as_in_context(ctx).reshape((-1, num_readings_in_gesture))
        # Comment this out to add some noise to the data
        data = data + 0.1*nd.mean(nd.abs(data)) * nd.random.normal(shape=data.shape)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
            loss.backward()
        trainer.step(data.shape[0])

        ##########################
        #  Keep a moving average of the losses
        ##########################
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)

    test_accuracy = evaluate_accuracy(testing_data, net)
    train_accuracy = evaluate_accuracy(training_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" %
          (e, moving_loss, train_accuracy, test_accuracy))

# After training, we can save the trained parameters to disk
net.save_params('ssd_%d-testing.params' % epochs)

Epoch 0. Loss: 2.703669393776569, Train_acc 0.7387755102040816, Test_acc 0.6571428571428571
Epoch 1. Loss: 2.5976105280859283, Train_acc 0.8408163265306122, Test_acc 0.8142857142857143
Epoch 2. Loss: 2.478828572953365, Train_acc 0.8816326530612245, Test_acc 0.8571428571428571
Epoch 3. Loss: 2.356115037171035, Train_acc 0.9306122448979591, Test_acc 0.9
Epoch 4. Loss: 2.230359458807729, Train_acc 0.963265306122449, Test_acc 0.9142857142857143
Epoch 5. Loss: 2.1088775781135576, Train_acc 0.9693877551020408, Test_acc 0.9142857142857143
Epoch 6. Loss: 1.990788033524717, Train_acc 0.9734693877551021, Test_acc 0.8857142857142857
Epoch 7. Loss: 1.878448196202987, Train_acc 0.9714285714285714, Test_acc 0.9285714285714286
Epoch 8. Loss: 1.771624543216809, Train_acc 0.9775510204081632, Test_acc 0.9285714285714286
Epoch 9. Loss: 1.6655612556574448, Train_acc 0.9775510204081632, Test_acc 0.9
Epoch 10. Loss: 1.566727924601048, Train_acc 0.9775510204081632, Test_acc 0.9285714285714286
Epoch 11. Loss:

Epoch 92. Loss: 0.05183837073639754, Train_acc 1.0, Test_acc 0.9428571428571428
Epoch 93. Loss: 0.05159497876067116, Train_acc 1.0, Test_acc 0.9428571428571428
Epoch 94. Loss: 0.051156354086510585, Train_acc 1.0, Test_acc 0.9428571428571428
Epoch 95. Loss: 0.04987028647319851, Train_acc 1.0, Test_acc 0.9428571428571428
Epoch 96. Loss: 0.047527554025583986, Train_acc 1.0, Test_acc 0.9428571428571428
Epoch 97. Loss: 0.047483288422397955, Train_acc 1.0, Test_acc 0.9428571428571428
Epoch 98. Loss: 0.04624384158835569, Train_acc 1.0, Test_acc 0.9428571428571428
Epoch 99. Loss: 0.04655477974745708, Train_acc 1.0, Test_acc 0.9428571428571428
Epoch 100. Loss: 0.04513527699145481, Train_acc 1.0, Test_acc 0.9428571428571428
Epoch 101. Loss: 0.04390019384034757, Train_acc 1.0, Test_acc 0.9428571428571428
Epoch 102. Loss: 0.04470597703953521, Train_acc 1.0, Test_acc 0.9428571428571428
Epoch 103. Loss: 0.043947642225778905, Train_acc 1.0, Test_acc 0.9428571428571428
Epoch 104. Loss: 0.0425708550747

In [11]:
# Load raw data from a known gesture (shutter on / off)
single_sample_reader = pd.read_csv('shutter.csv')

single_sample_matrix = single_sample_reader.as_matrix()
single_sample_data = np.asarray(single_sample_matrix[:,0:6], dtype=np.float64)
single_sample_data = single_sample_data.reshape(-1, num_datapoints_in_gesture, num_features_in_sample)
single_sample_data = single_sample_data.reshape(-1, num_readings_in_gesture)

# pass the data from a single gesture to the trained network to verify the result
result_num = int(net(mx.nd.array(single_sample_data)).argmax(axis=1).asscalar())

result_dict = { 
    1:'tv-poke', 
    5:'fan-one-circle', 
    6:'fan-two-circles', 
    2:'shutter-right-left', 
    0:'no-gesture', 
    3:'letter-m', 
    4:'letter-y'
    
}

print("The network's guess is: \n")

print(result_dict[result_num])

The network's guess is: 

shutter-right-left
